In [19]:
import pandas as pd
import numpy as np
import json
import pprint
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [24]:
data = pd.read_json('68_Berlin_Munich.json', orient = 'records', encoding = 'utf8')
data.head()
data.columns

Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
       'blog_url', 'category_code', 'competitions', 'created_at',
       'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds'],
      dtype='object')

In [37]:
comp = data[['name', 'offices', 'funding_rounds', 'total_money_raised', 'created_at', 'founded_year', 'number_of_employees', 'category_code', 'description', 'overview', 'tag_list']].copy()
comp


,name,offices,funding_rounds,total_money_raised,created_at,founded_year,number_of_employees,category_code,description,overview,tag_list
0,amiando,"[{'description': '', 'address1': 'Sandstrasse ...","[{'id': {'$numberInt': '1552'}, 'round_code': ...",$0,2008-02-13 15:29:13,{'$numberInt': '2006'},{'$numberInt': '76'},web,Online Event Registration,<p>amiando is a pioneer in online event regist...,"events, event-community, , ticketing, event-re..."
1,GetYourGuide,"[{'description': 'Headquarters', 'address1': '...","[{'id': {'$numberInt': '8275'}, 'round_code': ...",$16M,2008-08-05 01:07:53,{'$numberInt': '2009'},{'$numberInt': '75'},travel,Booking website for tours & activities,<p>GetYourGuide collects and categorizes all t...,"online-travel, web-2-0, tour-guide, travel, tr..."
2,ResearchGate,"[{'description': 'HQ', 'address1': 'Invalidens...","[{'id': {'$numberInt': '19574'}, 'round_code':...",$35M,2008-10-14 23:41:33,{'$numberInt': '2008'},{'$numberInt': '74'},social,The professional network for scientists.,<p>ResearchGate is the leading social network ...,"academia, research, social-network, scientific..."
3,Videoplaza,"[{'description': 'London HQ', 'address1': 'Wat...","[{'id': {'$numberInt': '2904'}, 'round_code': ...",€15.9M,2008-07-10 20:12:07,{'$numberInt': '2007'},{'$numberInt': '70'},advertising,Video Ad Management,<p>Videoplaza powers the video advertising bus...,"video-advertising, video, stockholm, london, o..."
4,My-Hammer,"[{'description': 'MY-HAMMER AG', 'address1': '...","[{'id': {'$numberInt': '6105'}, 'round_code': ...",$3.2M,2009-05-23 11:15:03,{'$numberInt': '2005'},{'$numberInt': '70'},web,Handyman job portal,<p>MyHammer is an online marketplace and direc...,"builder, plumber, painter, carpenter, gardenin..."
5,skobbler,"[{'description': 'Headquarters', 'address1': '...",[],$0,2009-05-24 20:43:02,{'$numberInt': '2008'},{'$numberInt': '70'},mobile,LBS and mobile navigation,<p>skobbler is a Berlin-based maps and navigat...,"skobbler, lbs, navigation, gps, mobile-social-..."
6,metaio,"[{'description': 'metaio GmbH', 'address1': 'I...",[],$0,2008-09-24 23:48:15,{'$numberInt': '2003'},{'$numberInt': '65'},software,,"<p>Founded in 2003, <strong>metaio</strong> is...","augmented-reality, mobile, advertising, marketing"
7,Geewa,"[{'description': 'World Headquarters', 'addres...","[{'id': {'$numberInt': '5134'}, 'round_code': ...",€1.5M,2009-02-20 10:12:37,{'$numberInt': '2005'},{'$numberInt': '60'},games_video,casual competitive multiplayer games,<p>Geewa develops and operates real-time casua...,"multiplayer-game, game, casual-game, flash-gam..."
8,Smart AdServer,"[{'description': 'Smart AdServer North Am', 'a...",[],$0,2009-04-14 12:45:14,{'$numberInt': '2001'},{'$numberInt': '60'},advertising,Premium ad serving solutions,<p>Smart AdServer SAS develops and markets one...,"ad-serving, mobile-advertising, tablet-adverti..."
9,GRIN Publishing,"[{'description': 'Headoffice Munich', 'address...","[{'id': {'$numberInt': '6491'}, 'round_code': ...",$0,2009-06-15 12:14:28,{'$numberInt': '1998'},{'$numberInt': '55'},software,SaaS Solution for Book Publishers,<p>GRIN Publishing is an online based publishi...,"publishing, academic, book, ebook, print-on-de..."


In [40]:
def get_coords(data):
    
    data = data['offices']

    coords = None
    if data[0]['latitude'] and data[0]['longitude']:
        coords = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "site": coords
    }

location = comp[["offices"]].apply(get_coords, result_type="expand", axis=1)


In [43]:
expanded = pd.concat([comp,location], axis=1)[["name","lat","lng", "site"]]
expanded.head()


,name,lat,lng,site
0,amiando,{'$numberDouble': '48.124827'},{'$numberDouble': '11.556452'},"{'type': 'Point', 'coordinates': [{'$numberDou..."
1,GetYourGuide,{'$numberDouble': '47.3912899'},{'$numberDouble': '8.5082891'},"{'type': 'Point', 'coordinates': [{'$numberDou..."
2,ResearchGate,{'$numberDouble': '52.5308516'},{'$numberDouble': '13.3842605'},"{'type': 'Point', 'coordinates': [{'$numberDou..."
3,Videoplaza,{'$numberDouble': '51.5212481'},{'$numberDouble': '-0.1020894'},"{'type': 'Point', 'coordinates': [{'$numberDou..."
4,My-Hammer,{'$numberDouble': '52.5092312'},{'$numberDouble': '13.3883606'},"{'type': 'Point', 'coordinates': [{'$numberDou..."


In [ ]:
def getFirst(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = df[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [ ]:
df_clean = pd.concat([df,first_office], axis=1)[["name","lat","lng", "oficina_principal","totalOffices"]]

In [ ]:
display(df_clean.shape)
df_clean.head()

In [ ]:
# Remove offices with unexisting lat,lng or both lat and lng data
#df_clean.dropna(axis=0, inplace=True)

# mongoimport --db companies --collection first_office --jsonArray ./data/oficinas.json
df_clean.to_json('./data/oficinas.json', orient="records")

In [ ]:
#https://docs.mongodb.com/manual/reference/operator/query/near/index.html
def findNear(geopoint, radio_max_meters=1000):
    return db.first_office.find({
        "oficina_principal": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })


# https://developers.google.com/maps/documentation/geocoding/intro
park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 10000
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))

In [ ]:
pd.DataFrame(findNear(park_avenue_con_21_street, 5000)).to_csv("./data/geoesp.csv")